<a href="https://colab.research.google.com/github/nishanthnandakumar/Lidar_Radar_Systems/blob/main/lidar_assignment3main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # load point clouds and more array/vector operations
import cv2 # load images
import plotly.graph_objects as go # visualize point clouds

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
root = "./gdrive/MyDrive/dataset/Record2/"

In [4]:
def rt_matrix(x=0, y=0, z=0, roll=0, pitch=0, yaw=0):
    """
        Calculates a 4x4 Transformation Matrix. Angels in radian!
    """
    c_y = np.cos(yaw)
    s_y = np.sin(yaw)
    c_r = np.cos(roll)
    s_r = np.sin(roll)
    c_p = np.cos(pitch)
    s_p = np.sin(pitch)
    
    rot = np.dot(np.dot(np.array([[c_y,-s_y,0],
                                     [s_y,c_y,0],
                                     [0,0,1]]),
                           np.array([[c_p,0,s_p],
                                     [0,1,0],
                                     [-s_p,0,c_p]])),
                            np.array([[1,0,0],
                                     [0,c_r,-s_r],
                                     [0,s_r,c_r]]))
    matrix = np.array([[0,0,0,x],
                     [0,0,0,y],
                     [0,0,0,z],
                     [0,0,0,1.]])
    matrix[0:3,0:3] += rot
    return matrix

In [5]:
#Calculate the 8 corners of the bounding box
def transfer_points(points, rot_t):
    """
        Calculates the transformation of a point cloud for a given transformation matrix.
    """
    points = np.concatenate([points, np.ones([1,points.shape[1]])])
    points[0:3,:] = np.dot(rot_t, points[0:4,:])[0:3, :]
    return points[0:3]

In [6]:
def make_boundingbox(label):
    """
        Calculates the Corners of a bounding box from the parameters.
    """
    corner = np.array([
        [+ label[3]/2, + label[4]/2, + label[5]/2],
        [+ label[3]/2, + label[4]/2, - label[5]/2],
        [+ label[3]/2, - label[4]/2, + label[5]/2],
        [+ label[3]/2, - label[4]/2, - label[5]/2],
        [- label[3]/2, + label[4]/2, + label[5]/2],
        [- label[3]/2, - label[4]/2, + label[5]/2],
        [- label[3]/2, + label[4]/2, - label[5]/2],
        [- label[3]/2, - label[4]/2, - label[5]/2],
    ])
    corner = transfer_points(corner.T, rt_matrix(yaw = label[6])).T
    corner = corner + label[0:3]
    return corner

In [7]:
bb1 = np.loadtxt(root + "Blickfeld/bounding_box/000007.csv")

In [8]:
bb2 = np.loadtxt(root + "/Blickfeld/bounding_box/000008.csv")

In [9]:
bb3 = np.loadtxt(root + "/Blickfeld/bounding_box/000009.csv")

In [10]:
blick = np.loadtxt(root + "Blickfeld/point_cloud/000009.csv")

In [11]:
def bb_pred(boundingbox1, boundingbox2):

  error = boundingbox2 - boundingbox1
  print(error)
  bbox_pred = boundingbox2 + error
  print(bbox_pred)

  return bbox_pred

In [12]:
bx_pred = bb_pred(bb1, bb2)

[-2.21813934e+00  2.05084107e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  7.69352143e-04]
[-1.73012363  5.51480356 -0.0705      4.304       1.94        1.459
 -0.0276375 ]


In [13]:
bb3

array([-2.96002472,  5.52995854, -0.0705    ,  4.304     ,  1.94      ,
        1.459     , -0.02840685])

In [14]:
bb1_cor = make_boundingbox(bx_pred)

In [15]:
bb1_cor

array([[ 0.4478595 ,  6.42496481,  0.659     ],
       [ 0.4478595 ,  6.42496481, -0.8       ],
       [ 0.39424958,  4.48570568,  0.659     ],
       [ 0.39424958,  4.48570568, -0.8       ],
       [-3.85449684,  6.54390145,  0.659     ],
       [-3.90810676,  4.60464232,  0.659     ],
       [-3.85449684,  6.54390145, -0.8       ],
       [-3.90810676,  4.60464232, -0.8       ]])

In [16]:
bb2_cor = make_boundingbox(bb3)

In [17]:
bb2_cor

array([[-0.781342  ,  6.43844388,  0.659     ],
       [-0.781342  ,  6.43844388, -0.8       ],
       [-0.83644388,  4.49922656,  0.659     ],
       [-0.83644388,  4.49922656, -0.8       ],
       [-5.08360557,  6.56069051,  0.659     ],
       [-5.13870744,  4.6214732 ,  0.659     ],
       [-5.08360557,  6.56069051, -0.8       ],
       [-5.13870744,  4.6214732 , -0.8       ]])

In [18]:
def minmaxbb(bounding_box):
  #Here we max and min for bounding box
  #Initialize the variables with some maximum and minimum values
  x_max = -10000
  x_min = 10000
  y_max = -10000
  y_min = 10000
  z_max = -10000
  z_min = 10000
  for corner in bounding_box:
    if corner[0] >= x_max:
      x_max = corner[0]
    if corner[0] <= x_min:
      x_min = corner[0]

    if corner[1] >= y_max:
      y_max = corner[1]
    if corner[1] <= y_min:
      y_min = corner[1]

    if corner[2] >= z_max:
      z_max = corner[2]
    if corner[2] <= z_min:
      z_min = corner[2]

  val = [x_max,x_min,y_max,y_min,z_max,z_min]
  return val

In [19]:
bb1_minmax = minmaxbb(bb1_cor)

In [20]:
bb1_minmax

[0.44785949866662556,
 -3.9081067585829494,
 6.543901450839497,
 4.485705678528381,
 0.6590000000000001,
 -0.7999999999999999]

In [21]:
bb2_minmax = minmaxbb(bb2_cor)

In [22]:
bb2_minmax

[-0.7813420041243568,
 -5.13870744108962,
 6.560690509017719,
 4.499226563990383,
 0.6590000000000001,
 -0.7999999999999999]

In [23]:
def bb_iou(boxA, boxB):
  # determine the (x, y)-coordinates of the intersection cuboid
  xA = min(boxA[0], boxB[0])
  xB = max(boxA[1], boxB[1])
  yA = min(boxA[2], boxB[2])
  yB = max(boxA[3], boxB[3])
  zA = min(boxA[4], boxB[4])
  zB = max(boxA[5], boxB[5])
  # compute the volume of intersection cuboid
  interVolume = (xA - xB) * (yA - yB) * (zA - zB)
  # compute the Volume of both the prediction and ground-truth
  # cuboids
  boxAVolume = (boxA[0] - boxA[1] ) * (boxA[2] - boxA[3] ) * (boxA[4] - boxA[5] )
  boxBVolume = (boxB[0] - boxB[1] ) * (boxB[2] - boxB[3] ) * (boxB[4] - boxB[5] )
  # compute the intersection over union by taking the intersection
  # volume and dividing it by the sum of prediction + ground-truth
  # volumes - the interesection volume
  iou = interVolume / float(boxAVolume + boxBVolume - interVolume)
  # return the intersection over union value
  return iou

In [24]:
iou = bb_iou(bb1_minmax, bb2_minmax)

In [25]:
iou

0.5532970145861178

In [26]:
data = [ go.Scatter3d(x = blick[:,0],
                     y = blick[:,1],
                     z = blick[:,2],
                     text=np.arange(8),
                    mode='markers', type='scatter3d',
                    marker={
                        'size': 2,
                        'color': "green",
                        'colorscale':'rainbow',
}),

        go.Scatter3d(x = bb1_cor[:,0],
                     y = bb1_cor[:,1],
                     z = bb1_cor[:,2],
                     text=np.arange(8),
                    mode='markers', type='scatter3d',
                    marker={
                        'size': 2,
                        'color': "blue",
                        'colorscale':'rainbow',
}),
        
        go.Scatter3d(x = bb2_cor[:,0],
                     y = bb2_cor[:,1],
                     z = bb2_cor[:,2],
                     text=np.arange(8),
                    mode='markers', type='scatter3d',
                    marker={
                        'size': 2,
                        'color': "red",
                        'colorscale':'rainbow',
})
]
layout = go.Layout(
    scene={
        'xaxis': {'range': [-25, 25], 'rangemode': 'tozero', 'tick0': -5},
        'yaxis': {'range': [-25, 25], 'rangemode': 'tozero', 'tick0': -5},
        'zaxis': {'range': [-25, 25], 'rangemode': 'tozero'}
    }
)
go.Figure(data=data, layout = layout)

